In [2]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from pandas.io.json import json_normalize   
import pandas as pd  
import time
import json
import re


In [3]:
#################################################################################
#################################################################################
############################    PATH SETTINGS   #################################

# Building URL. The url is build as site_search + date_path + model_path
# In order to use the script for each city is useful adding name_city to the path
city_name = "trento"

years = ["2011","2012","2013","2014","2015","2016","2017","2018","2019","2020"]

In [4]:
year = 2020

# Search period settings gg.mm.yyyy
date_from = "from/01.01.{}/".format(year)
date_to = "to/31.12.{}/".format(year)

date_path = date_from + date_to

# Search model settings
model_path = "model/articolo/"

# Building URL. The url is build as site_search + date_path + model_path
my_url = "http://www.{}today.it/search/{}{}".format(city_name, model_path,date_path)

print("Internet site : ", my_url)

#################################################################################
#################################################################################
############################        PARSING       ###############################

# Opening up connection, grabbing the page
UClient =  uReq(my_url)
time.sleep(3)
page_html = UClient.read()

# html parser
page_soup = soup(page_html,"lxml")

print("Parsing page...")

# Cerco il numero di pagine per ciclare su ognuna di esse
temp = page_soup.find("div",{"class", "form-group search-input"})
n_pages = temp.find_all("strong")[1].get_text()

# Creo il database
columns = ['title','subtitle',  'link', 'timestamp', 'text', 'keywords' ,'address']
df = pd.DataFrame(columns=columns)

#################################################################################
#################################################################################
############################    STEALING ARTICLES   #############################

print("Cicling on {} pages".format(n_pages))

url_search = '{}pag/'.format(my_url)



Internet site :  http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/
Parsing page...
Cicling on 111 pages


In [5]:

# Ciclo sulle pagine e ricavo i link agli articoli
for i in range(1,int(n_pages)+1):
    try:
        url_complete = url_search + str(i) #creo il link
        print(url_complete)

        UClient =  uReq(url_complete)
        #time.sleep(2)
        html = UClient.read()

        page_soup = soup(html,"lxml")
        stories = page_soup.find("ul",{"class","u-unstyled search-results"})
        articles = stories.findAll("li",{"class","result-item nw_result_articolo"})
        links = []

        for art in articles:
            if(art.article.div.header.p.span.get_text()=="Articolo"):
                link = 'http://www.trentotoday.it{}'.format(art.a['href'])
                #df = df.append({'title':'', 'link':link, 'timestamp':'' ,'text':'', 'keywords':'', 'address':'', 'gps':''},ignore_index = True)
                links.append(link)
    except:
        print('Eccezione alla pagine', i)

http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/1
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/2
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/3
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/4
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/5
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/6
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/7
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/8
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/9
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/10
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/pag/11
http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2

,title,subtitle,link,timestamp,text,keywords,address


'http://www.trentotoday.it/attualita/distruggere-nido-rondini.html'

In [12]:
titles = []
subtitles = []
timestamps = []
texts = []
keywords = []
addresses = []

for link in links:
    UClient =  uReq(link)
    #time.sleep(2)
    html = UClient.read()

    page_soup = soup(html,"lxml")
    try:
        #get title
        title = page_soup.find("h1", ("class","entry-title")).get_text().strip()
        titles.append(title)
        print(title)
        #get subtitle
        subtitle = page_soup.find("p",("class", "summary entry-summary")).get_text().strip()
        subtitles.append(subtitle)
        #get timestamp
        timestamp = page_soup.find("time", ("class", "datestamp")).get_text().strip()
        timestamps.append(timestamp)
        #get text
        p = page_soup.find("div", ("class", "entry-content-body")).findAll("p")
        text= ""
        for testo in p:
            text += testo.get_text().strip() + "\n" 
        texts.append(text)
        #get keywords
        li = page_soup.findAll("li", ("class", "tag-item"))
        parolechiave = []
        for a in li:
            parolechiave.append(a.get_text().strip())
        keywords.append(parolechiave[1:])
        print(parolechiave[1:])
        #get address
        span_address = page_soup.findAll("span", ("class","entry-label"))
        address = []
        for span in span_address:
            #address.append(span.a.get_text().strip())
            a = span.find("a")
            if(a!= None):
                address.append(a.get_text().strip())
            else:
                address.append(span.get_text().splitlines()[1].strip()) #accorgimenti causati da una lettura sbagliata della stringa
        addresses.append(address[1:])
        print(address[1:])
    except:
        print('\nEccezione alla pagine', link)

# Definisco il dataset
df.append({'title':titles,'subtitle':subtitles, 'link':links, 'timestamp':timestamps ,'text':'', 'keywords':keywords, 'address':addresses},ignore_index = True)
df.to_csv("{}/{}.csv".format(city_name,year),header = True)

Bimbo di 6 anni cade dalla seggiovia: medico lo afferra per la giacca e lo salva
['incidenti sci']
['Brentonico', "Localita' Polsa"]
A22, nessun aumento sui pedaggi nel 2020
['a22', 'pedaggi']
[]
Samantha Cristoforetti lascia l'Aeronautica
['Samantha Cristoforetti', 'Argomenti:', 'esercito', 'samantha cristoforetti']
['Centro storico', 'Piazza Dante']
Meteo, weekend di sole e temperature sopra la media
['meteo', 'previsioni']
['Centro storico', 'Piazza del Duomo']
Inseguito dai carabinieri tenta di buttare la droga nel lago
['carabinieri', 'droga']
['Tonadico']
Ticket sanitario, nuove regole e esenzioni per tutti (non c'entra il reddito)
['salute', 'sanità', 'ticket']
[]
Saldi invernali 2020 a Trento: al via i primi di gennaio
['promozioni', 'saldi']
[]
Tuffo del Primo dell'Anno a Riva del Garda: 125 nelle acque del lago
['riva del garda', 'tuffo capodanno', 'tuffo merla']
['Centro storico', 'Piazza Catena']
Capodanno: un successo le feste in piazza a Trento e Rovereto
['capodanno']
['

Blitz dei carabinieri nell'oratorio che ospita i migranti: 40 grammi di hashish, struttura chiusa
['Argomenti:', 'migranti', 'senza dimora', 'stranieri']
['Centro storico', 'Via Carlo Rosmini']


Page we are analysing :  http://www.trentotoday.it/search/model/articolo/from/01.01.2020/to/31.12.2020/
Parsing page...
Cicling on 112 pages
Getting links...
Links obtained!
Recuperato il corpo di Daniele Catorci
Cade nel fiume: ripescato vivo un chilometro e mezzo a valle, è grave
La Provincia: "L'orso del video sarà catturato e munito di radiocollare"
Sagra della braciola d'orso con il patrocinio del Trentino: il Comune di Bugliano colpisce ancora
Dramma sul Gran Paradiso: individuato lo zaino di Daniele, nessuna speranza di trovarlo vivo
Parkinson: al S. Chiara impiantato in un paziente uno stimolatore del cervello "profondo"
L'orso si avvicina all'escursionista e lo segue: l'incontro è avvenuto in Val di Non
Bonus baby sitter anche ai nonni
Sacchetti della differenziata finiti? Ecco dove ritirarli gratis
Riva del Garda: 1.200 bottiglie di spumante calate nel lago, ecco perché
Coronavirus: contagi anche nella Nazionale azzurra di sci in ritiro allo Stelvio
Nido distrutto, balestrucc

In [4]:
df = pd.DataFrame(list(zip(titles, subtitles, links, timestamps,texts, keywords, addresses)), columns=columns)

In [7]:
df.to_csv("{}/{}.csv".format(city_name,year),header = True)